# Assignment 4 - Hypothesis Testing

This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%. 

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import ttest_ind

In [2]:
# Use this dictionary to map state names to two letter acronyms
states1 = {'Ohio': 'OH', 'Kentucky': 'KY', 'American Samoa': 'AS', 'Nevada': 'NV', 'Wyoming': 'WY', 'National': 'NA', 'Alabama': 'AL', 'Maryland': 'MD', 'Alaska': 'AK', 'Utah': 'UT', 'Oregon': 'OR', 'Montana': 'MT', 'Illinois': 'IL', 'Tennessee': 'TN', 'District of Columbia': 'DC', 'Vermont': 'VT', 'Idaho': 'ID', 'Arkansas': 'AR', 'Maine': 'ME', 'Washington': 'WA', 'Hawaii': 'HI', 'Wisconsin': 'WI', 'Michigan': 'MI', 'Indiana': 'IN', 'New Jersey': 'NJ', 'Arizona': 'AZ', 'Guam': 'GU', 'Mississippi': 'MS', 'Puerto Rico': 'PR', 'North Carolina': 'NC', 'Texas': 'TX', 'South Dakota': 'SD', 'Northern Mariana Islands': 'MP', 'Iowa': 'IA', 'Missouri': 'MO', 'Connecticut': 'CT', 'West Virginia': 'WV', 'South Carolina': 'SC', 'Louisiana': 'LA', 'Kansas': 'KS', 'New York': 'NY', 'Nebraska': 'NE', 'Oklahoma': 'OK', 'Florida': 'FL', 'California': 'CA', 'Colorado': 'CO', 'Pennsylvania': 'PA', 'Delaware': 'DE', 'New Mexico': 'NM', 'Rhode Island': 'RI', 'Minnesota': 'MN', 'Virgin Islands': 'VI', 'New Hampshire': 'NH', 'Massachusetts': 'MA', 'Georgia': 'GA', 'North Dakota': 'ND', 'Virginia': 'VA'}
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
states1 = pd.Series(states1).reset_index() 
#states_df = pd.DataFrame(states_df)
#states_df = states_df.reset_index()
states1.rename(columns={'index':'State'}, inplace = True)
#states_df

Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'.
    

In [4]:
def get_list_of_university_towns():
    with open('university_towns.txt') as u_towns:
        #u_t_s = []
        #for line in u_towns:
            #u_t_s.append(line[:-1])
    
        s_t = []
        for line in u_towns: #for line in u_t_s:
            if line[-6:-2] == 'edit': #if line[-6:] == '[edit]':
                state = line[0:-7] # state = line[:-6]
            elif '(' in line:
                town = line[:(line.index('(')-1)]
                s_t.append([state,town])
            else:
                town = line.rstrip()
                s_t.append([state,town])
    df = pd.DataFrame(s_t, columns = ['State', 'RegionName']) #.set_index('State')
    #df.append(states_df, ignore_index = True)
    df = pd.merge(df, states1, how = 'left', left_on = 'State', right_on = 'State')
    df = df.rename(columns={0:'Acronyms'}).set_index('Acronyms')
    return df
get_list_of_university_towns()

,State,RegionName
Acronyms,,
AL,Alabama,Auburn
AL,Alabama,Florence
AL,Alabama,Jacksonville
AL,Alabama,Livingston
AL,Alabama,Montevallo
...,...,...
WI,Wisconsin,River Falls
WI,Wisconsin,Stevens Point
WI,Wisconsin,Waukesha


Returns the year and quarter  of the  recession start time as a string value in a format such as 2005q3

In [5]:
Qu_GDP = pd.DataFrame(pd.read_excel('gdplev.xls', skiprows=5).drop(0).drop(1)).rename(columns={'Unnamed: 0':'Year','Unnamed: 3':'Hi', 'Unnamed: 4':'Year&Quarters','Unnamed: 7':'Ho'}).reset_index()
Qu_GDP = Qu_GDP[['Year', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars', 'Year&Quarters', 'GDP in billions of current dollars.1', 'GDP in billions of chained 2009 dollars.1']]
Qu_GDP

,Year,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,Year&Quarters,GDP in billions of current dollars.1,GDP in billions of chained 2009 dollars.1
0,1929.0,104.6,1056.6,1947q1,243.1,1934.5
1,1930.0,92.2,966.7,1947q2,246.3,1932.3
2,1931.0,77.4,904.8,1947q3,250.1,1930.3
3,1932.0,59.5,788.2,1947q4,260.3,1960.7
4,1933.0,57.2,778.3,1948q1,266.2,1989.5
...,...,...,...,...,...,...
273,NaN,NaN,NaN,2015q2,17998.3,16374.2
274,NaN,NaN,NaN,2015q3,18141.9,16454.9
275,NaN,NaN,NaN,2015q4,18222.8,16490.7
276,NaN,NaN,NaN,2016q1,18281.6,16525.0


In [6]:
def get_recession_start():
    re_st = Qu_GDP.loc[:,['Year&Quarters', 'GDP in billions of chained 2009 dollars.1']].iloc[212:].reset_index()
    re_st = re_st[['Year&Quarters', 'GDP in billions of chained 2009 dollars.1']]
    recessionstarts = []
    for item in range(len(re_st)-4):
        if ((re_st.iloc[item][1] > re_st.iloc[item+1][1]) & (re_st.iloc[item+1][1] > re_st.iloc[item+2][1])):
            recessionstarts.append(re_st.iloc[item][0])
    res = recessionstarts[0]
    return res
get_recession_start()

'2008q2'

Returns the year and quarter of the recession start time as a string value in a format such as 2005q3

In [7]:
def get_recession_end():
    re_st = Qu_GDP.loc[:,['Year&Quarters', 'GDP in billions of chained 2009 dollars.1']].iloc[212:].reset_index()
    re_st = re_st[['Year&Quarters', 'GDP in billions of chained 2009 dollars.1']]
    recessionends = []
    for item in range(len(re_st)-4):
        if ((re_st.iloc[item][1] > re_st.iloc[item+1][1]) & (re_st.iloc[item+1][1] > re_st.iloc[item+2][1]) & (re_st.iloc[item+2][1] < re_st.iloc[item+3][1]) & (re_st.iloc[item+3][1] < re_st.iloc[item+4][1])):
            recessionends.append(re_st.iloc[item+4][0])
    res = recessionends[0]
    return res
get_recession_end()

'2009q4'

In [8]:
def get_recession_bottom():
    re_st = Qu_GDP.loc[:,['Year&Quarters', 'GDP in billions of chained 2009 dollars.1']].iloc[212:].reset_index()
    re_st = re_st[['Year&Quarters', 'GDP in billions of chained 2009 dollars.1']]
    recessionends = []
    for item in range(len(re_st)-4):
        if ((re_st.iloc[item][1] > re_st.iloc[item+1][1]) & (re_st.iloc[item+1][1] > re_st.iloc[item+2][1]) & (re_st.iloc[item+2][1] < re_st.iloc[item+3][1]) & (re_st.iloc[item+3][1] < re_st.iloc[item+4][1])):
            recessionends.append(re_st.iloc[item+2][0])
    res = recessionends[0]
    return res
get_recession_bottom()

'2009q2'

Converts the housing data to quarters and returns it as mean values in a dataframe. This dataframe should be a dataframe with columns for 2000q1 through 2006q3, and should have a multi-index in the shape of ["State","RegionName"].

Note: Quarters are defined in the assignment description, they are not arbitrary three months periods.

The resulting dataframe should have 67 columns, and 10,730 rows

In [9]:
def convert_housing_data_to_quarters():
    ho_da = pd.DataFrame(pd.read_csv('City_Zhvi_AllHomes.csv'))
    ho_da1 = ho_da.iloc[:,51:252]
    ho_da2 = ho_da.iloc[:,1:3]
    ho_da = pd.merge(ho_da2, ho_da1, how='outer', left_index = True, right_index = True).set_index(['State', 'RegionName'])
    #ho_da = ho_da.drop(ho_da.columns[[0] + list(range(3,51))], axis=1)
    #ho_da['Q1'] = (ho_da['2000-01'].map(float)+ho_da['2000-02'].map(float)+ho_da['2000-03'].map(float)+ho_da['2000-04'].map(float))/4
    ho_da = ho_da.rename(pd.to_datetime, axis='columns').resample('Q', axis=1).mean().rename(lambda x: str(x.to_period('Q')).lower(), axis='columns')
    return ho_da
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
CA,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
IL,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
PA,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
AZ,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WI,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
NY,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
WI,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


First creates new data showing the decline or growth of housing  prices between the recession start and the recession bottom. Then runs a ttest comparing the university towns values to the non-university towns values, return whether the alternative hypothesis (that the two groups are the same) is true or no as well as the p-value of the confidence.

Return the tuple (different, p, better) where different=True if the t-test is True at a p<0.01 (we reject the null hypothesis), or different =False if otherwise (we cannot reject the null hypothesis). The variable  p should be equal to the exact p value returned from scipy.stats.ttest_ind(). The value for better should be either "university town" or "non-university town" depending on which has a lower mean price ratio (wich is equivalent to a reduced market loss).

In [10]:
def run_ttest():
    UTowns = get_list_of_university_towns()
    Re_Start = get_recession_start()
    Re_Bottom = get_recession_bottom()
    HDTQ = convert_housing_data_to_quarters().reset_index().replace({'State':states}).set_index(['State', 'RegionName'])
    
    #rec_st = HDTQ.columns[HDTQ.columns.get_loc(Re_Start)]
    HDTQ['Rate'] =  HDTQ[Re_Start] / HDTQ[Re_Bottom]
    HDTQ = HDTQ[[Re_Bottom, Re_Start, 'Rate']]
    HDTQ = HDTQ.reset_index()
    
    #UTHD = pd.merge(UTowns, HDTQ, how='inner', on=['State','RegionName'])
    #UTHD = pd.merge(UTowns, HDTQ, how='left', left_index = True, right_index = True)
    UTHD = pd.merge(UTowns, HDTQ, how = 'inner', left_on = (['State','RegionName']), right_on = (['State','RegionName']))
    UTHD['UV'] = True
    Verif = pd.merge(HDTQ, UTHD, how='outer', on=['State','RegionName',Re_Bottom, Re_Start, 'Rate'])
    Verif['UV'] = Verif['UV'].fillna(False)
     
    UnivT = Verif[Verif['UV'] == True]
    NonUnivT = Verif[Verif['UV'] == False]
    
    r,p = ttest_ind(UnivT['Rate'].dropna(), NonUnivT['Rate'].dropna())
    different = True if p<0.01 else False
    better = "university town" if UnivT['Rate'].mean() < NonUnivT['Rate'].mean() else "non-university town"
    
    return (different, p, better)

run_ttest()

(True, 0.002724063704761454, 'university town')